# Approach 1

In [1]:
import os
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score 
from matplotlib import pyplot as plt


tile_size = 19
feature_count = 3
input_count = tile_size * tile_size * feature_count
DATA_FILES_ROOT = "DataFiles/"
PRED_FILES_ROOT = "Predictions/"

In [2]:
def load_data(dataset_name: str) -> np.array:
    data_path = DATA_FILES_ROOT + dataset_name + "_data/"
    data_files = os.listdir(data_path)
    data_array = [None] * len(data_files)
    for i, file in enumerate(data_files):
        if i % 1000 == 0:
            print(f"loading {i}. file")
        data = np.loadtxt(data_path + file, skiprows=1, delimiter=',')
        data_array[i] = data.flatten()
    return np.array(data_array)


def load_labels(dataset_name: str) -> np.array:
    labels_file = DATA_FILES_ROOT + dataset_name + "_truth.csv"
    labels = np.loadtxt(labels_file, delimiter=',')
    labels_array = [None] * len(labels)
    for i, label in enumerate(labels):
        labels_array[i] = label[2]
    return np.array(labels_array)

def load_datasets(datasets: list) -> (np.array, np.array):
    data = []
    labels = np.array([])
    for dataset in datasets:
        if (len(data) == 0):
            data = load_data(dataset)
        else:
            data = np.concatenate((data, load_data(dataset)), axis=0)
        labels = np.concatenate((labels, load_labels(dataset)), axis=0)
    return data, labels

datasets = ["fruit_02", "fruit_03", "fruit_04"]
data, labels = load_datasets(datasets)
print("done loading datasets")

loading 0. file
loading 1000. file
loading 2000. file
loading 3000. file
loading 4000. file
loading 5000. file
loading 6000. file
loading 7000. file
loading 8000. file
loading 9000. file
loading 10000. file
loading 0. file
loading 1000. file
loading 2000. file
loading 3000. file
loading 4000. file
loading 5000. file
loading 6000. file
loading 7000. file
loading 0. file
loading 1000. file
loading 2000. file
loading 3000. file
loading 4000. file
loading 5000. file
loading 6000. file
loading 7000. file
loading 8000. file
done loading datasets


In [3]:
print(data.shape)
print(labels.shape)

(26913, 1080)
(26913,)


# Normalization and Split

In [4]:
data_mean = np.mean(data, axis=0)
data_std = np.std(data, axis=0)
data -= data_mean
data /= data_std

In [5]:
train_X, test_X, train_y, test_y = train_test_split(data, labels, test_size=0.3, random_state=42)

print(len(train_X))
print(np.sum(train_y))

18839
18561.0


# Support Vector Machine

In [6]:
model_svm = svm.SVC(C=1000, gamma=1/10000, kernel='rbf')
#model_svm = svm.SVC(kernel='rbf')
model_svm.fit(train_X, train_y)

print(f"SVM Train accuracy: {model_svm.score(train_X, train_y)}")
print(f"Actual percentage of true points: {np.sum(train_y)/len(train_y)}")

SVM Train accuracy: 1.0
Actual percentage of true points: 0.9852433780986252


In [15]:
print(f"Cross validation scores: {cross_val_score(model_svm, train_X, train_y, cv=3)}")

Cross validation scores: [0.99140127 0.99124204 0.9906036 ]


In [ ]:
print(f"SVM Test accuracy: {model_svm.score(test_X, test_y)}")
print(f"Actual percentage of true points: {np.sum(test_y)/len(test_y)}")

# Random Forest

In [7]:
model_rfc = RandomForestClassifier(n_estimators=20, max_depth=10)
model_rfc.fit(train_X, train_y)

print(f"RFC Train accuracy: {model_svm.score(train_X, train_y)}")

RFC Train accuracy: 1.0


In [25]:
print(f"Cross validation scores: {cross_val_score(model_rfc, train_X, train_y, cv=3)}")
print(f"RFC Test accuracy: {model_rfc.score(test_X, test_y)}")

Cross validation scores: [0.99442675 0.99458599 0.99331104]
RFC Test accuracy: 0.9946742630666336


# Confusion Matrix

In [8]:
#SVM
train_y_svm_prediction = cross_val_predict(model_svm, train_X, train_y, cv=3)
confusion_svm = confusion_matrix(train_y, train_y_svm_prediction)
precision_svm = precision_score(train_y, train_y_svm_prediction)
recall_svm = recall_score(train_y, train_y_svm_prediction)

print(confusion_svm)
print(precision_svm)
print(recall_svm)

[[  166   112]
 [   53 18508]]
0.9939849624060151
0.9971445504013793


In [21]:
#RFC
train_y_rfc_prediction = cross_val_predict(model_rfc, train_X, train_y, cv=3)
confusion_rfc = confusion_matrix(train_y, train_y_rfc_prediction)
precision_rfc = precision_score(train_y, train_y_rfc_prediction)
recall_rfc = recall_score(train_y, train_y_rfc_prediction)

print(confusion_rfc)
print(precision_rfc)
print(recall_rfc)

[[  182    96]
 [    6 18555]]
0.994852822904938
0.9996767415548731


# Prediction

In [12]:
def load_indices(dataset_name: str) -> list:
    labels_file = DATA_FILES_ROOT + dataset_name + "_truth.csv"
    labels = np.loadtxt(labels_file, delimiter=',')
    indices = [0] * len(labels)
    for i, label in enumerate(labels):
        indices[i] = int(label[1])
    return indices

def make_prediction(model, dataset_name: str, export: bool = False) -> np.array:
    data = load_data(dataset_name)
    data -= data_mean
    data /= data_std
    labels = load_labels(dataset_name)
    prediction = model.predict(data)
    
    print(f"Precision on this dataset: {precision_score(labels, prediction)}")
    print(f"Recall on this dataset: {recall_score(labels, prediction)}")
    
    mistakes = 0
    zeroes = 0
    for i, label in enumerate(prediction):
        if round(prediction[i]) == 0:
            zeroes += 1
        if labels[i] != round(prediction[i]):
            mistakes += 1
    print(f"Made {mistakes} mistakes!")
    print(f"There were total of {zeroes} zeroes!")
    #print(f"Model accuracy: {model.score(data, labels)}")
    if export:
        export_prediction(dataset_name, prediction)
    return prediction

def export_prediction(dataset_name: str, prediction: np.array) -> None:
    indices = load_indices(dataset_name)
    prediction_file = PRED_FILES_ROOT + dataset_name + "_prediction.csv"
    with open(prediction_file, 'w') as file:
        for i, point_index in enumerate(indices):
            p = round(prediction[i])
            print(f"{i},{point_index},{int(p)}", file=file)
            
            
make_prediction(model_rfc, "fruit_01", True)

loading 0. file
loading 1000. file
loading 2000. file
loading 3000. file
loading 4000. file
loading 5000. file
loading 6000. file
loading 7000. file
loading 8000. file
loading 9000. file
loading 10000. file
loading 11000. file
loading 12000. file
Precision on this dataset: 0.9989469420818144
Recall on this dataset: 0.9998378466028863
Made 15 mistakes!
There were total of 59 zeroes!


array([1., 1., 1., ..., 1., 1., 1.])